In [1]:
import os
import csv
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../input_data_clean/bank_tract_clean_WITH_CENSUS.csv')

In [3]:
hmda_vars = ['num_approved_loans', 
             'num_applications',
             'sum_approved_loans',
             'sum_all_loan_amount'
             #'median__all_income',
             #'sum_rate_spread',
             #'mean_LTV'
            ]

In [4]:
data['pct_minority'] = (data['Tot.Pop'] - data['Tot.WhitePop'])/data['Tot.Pop'] * 100
#data['pct_minority'] = data['pct_minority'].apply(lambda x: "{:.2f%}".format(x))

In [5]:
# finding the median income quintiles
# sort the median__all_income variable in ascending order
data.sort_values(by='pct_minority', inplace=True)

# divide the sorted median__all_income variable into 5 equal parts, or quintiles
data['pct_minority_Quintiles'] = pd.qcut(data['pct_minority'], q=5, labels=False)

# create a new variable called "Median_Income_Quintiles" based on the quintiles
data['pct_minority_Quintiles'] = pd.cut(data['pct_minority'], bins=5, labels=['20', '40', '60', '80', '100'])

In [6]:
pd.set_option('display.max_rows', 100)

In [7]:
Quintiles_loan_table = data.groupby('which_bank')[hmda_vars].mean()
#Quintiles_loan_table['loan_approvel_rate'] = Quintiles_loan_table['num_approved_loans']/Quintiles_loan_table['num_applications']
#Quintiles_loan_table = Quintiles_loan_table.sort_values(by='loan_approvel_rate', ascending=False)
Quintiles_loan_table

,num_approved_loans,num_applications,sum_approved_loans,sum_all_loan_amount
which_bank,,,,
All Other Banks,54.401735,64.671093,2.576171e+07,2.963290e+07
Bank of West,2.394889,3.379333,1.312934e+06,1.767377e+06


In [14]:
Quintiles_loan_table3 = data.groupby(['which_bank', "pct_minority_Quintiles"])[hmda_vars].sum()
Quintiles_loan_table3['loan_pct_in_minority_group'] = Quintiles_loan_table3['num_approved_loans']/Quintiles_loan_table3['num_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table3['$loan_pct_in_minority_group'] = Quintiles_loan_table3['sum_approved_loans']/Quintiles_loan_table3['sum_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table3['loan_approval_rate'] = Quintiles_loan_table3['num_approved_loans']/Quintiles_loan_table3['num_applications']
Quintiles_loan_table3

num_approved_loans  num_applications  \
which_bank      pct_minority_Quintiles                                         
All Other Banks 20                                  161652            191003   
                40                                  185413            220696   
                60                                   96476            115442   
                80                                   51543             61692   
                100                                  12862             14998   
Bank of West    20                                    3135              4432   
                40                                    3786              5268   
                60                                    1984              2797   
                80                                    1466              2130   
                100                                    406               580   

                                        sum_approved_loans  \
which_bank      pct_minority_Quintiles                       
All Other Banks 20                            7.721641e+10   
                40                            8.600472e+10   
                60                            4.561390e+10   
                80                            2.488772e+10   
                100                           6.813170e+09   
Bank of West    20                            1.930785e+09   
                40                            2.067180e+09   
                60                            9.524000e+08   
                80                            7.263700e+08   
                100                           2.314700e+08   

                                        sum_all_loan_amount  \
which_bank      pct_minority_Quintiles                        
All Other Banks 20                             8.853590e+10   
                40                             9.873043e+10   
                60                             5.254122e+10   
                80                             2.910847e+10   
                100                            7.765210e+09   
Bank of West    20                             2.599130e+09   
                40                             2.742040e+09   
                60                             1.282095e+09   
                80                             9.996400e+08   
                100                            3.302900e+08   

                                        loan_pct_in_minority_group  \
which_bank      pct_minority_Quintiles                               
All Other Banks 20                                        0.318246   
                40                                        0.365025   
                60                                        0.189934   
                80                                        0.101473   
                100                                       0.025322   
Bank of West    20                                        0.290897   
                40                                        0.351304   
                60                                        0.184096   
                80                                        0.136030   
                100                                       0.037673   

                                        $loan_pct_in_minority_group  \
which_bank      pct_minority_Quintiles                                
All Other Banks 20                                         0.321018   
                40                                         0.357555   
                60                                         0.189634   
                80                                         0.103468   
                100                                        0.028325   
Bank of West    20                                         0.326797   
                40                                         0.349883   
                60                                         0.161200   
                80        

In [ ]:
# 18.9% is the national average for hispanic/latino population
# Here, we consider anything over the national average of 18.9% a high hispanic population
# Hispanic pop in AZ = 32.3%, in CA = 39.4%
data['high_hisp_pop'] = ((data['Tot.Hispanic/Latino']/data['Tot.Pop']) > 0.189)
high_hispanic= data[data['high_hisp_pop']]

In [ ]:
# finding the median income quintiles
# sort the median__all_income variable in ascending order
data.sort_values(by='median__all_income', inplace=True)

# divide the sorted median__all_income variable into 5 equal parts, or quintiles
data['Median_Income_Quintiles'] = pd.qcut(data['median__all_income'], q=5, labels=False)

# create a new variable called "Median_Income_Quintiles" based on the quintiles
data['Median_Income_Quintiles'] = pd.cut(data['Median_Income_Quintiles'], bins=5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])

In [ ]:
Quintiles_loan_table = data.groupby('Median_Income_Quintiles')[hmda_vars].mean()
Quintiles_loan_table['loan_approvel_rate'] = Quintiles_loan_table['num_approved_loans']/Quintiles_loan_table['num_applications']
Quintiles_loan_table = Quintiles_loan_table.sort_values(by='loan_approvel_rate', ascending=False)
Quintiles_loan_table